### loading packages

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob

### specifiying directories

In [4]:
path = os.getcwd()
home_path = os.path.dirname(path)
data_folder = f'{home_path}\\Data'
gis_folder = f'{home_path}\\GIS'

### loading needed geospatial layers

# 0. Dataset 
## 0.1 - P

All data is downloaded from:  https://www.ncei.noaa.gov/cdo-web/. The raw data file is too large to be stored, individual station data can be found [here](https://github.com/Daafip/ENVM1502-Catchment-model/tree/main/Data/P/data%20per%20station) in the repo. 

Initially the mean over the whole area was taken